## Deliverable 2. Create a Customer Travel Destinations Map.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chuy,UY,-33.6971,-53.4616,63.95,94,2,4.68,clear sky
1,1,Yellowknife,CA,62.4560,-114.3525,34.41,91,100,16.11,overcast clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,39.31,87,20,6.91,few clouds
3,3,Strezhevoy,RU,60.7333,77.5889,13.33,84,49,9.13,scattered clouds
4,4,Padang,ID,-0.9492,100.3543,86.05,62,97,4.97,overcast clouds


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 75
What is your desired maximum temperature for your trip? 90


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Padang,ID,-0.9492,100.3543,86.05,62,97,4.97,overcast clouds
9,9,Cockburn Town,TC,21.4612,-71.1419,78.19,80,100,21.32,overcast clouds
11,11,Sao Joao Da Barra,BR,-21.6403,-41.0511,75.72,84,6,8.37,clear sky
15,15,Hithadhoo,MV,-0.6000,73.0833,83.34,68,89,7.61,overcast clouds
17,17,Pontianak,ID,-0.0333,109.3333,85.98,74,75,17.27,broken clouds


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_city_df = preferred_cities_df.dropna()
clean_city_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Padang,ID,-0.9492,100.3543,86.05,62,97,4.97,overcast clouds
9,9,Cockburn Town,TC,21.4612,-71.1419,78.19,80,100,21.32,overcast clouds
11,11,Sao Joao Da Barra,BR,-21.6403,-41.0511,75.72,84,6,8.37,clear sky
15,15,Hithadhoo,MV,-0.6000,73.0833,83.34,68,89,7.61,overcast clouds
17,17,Pontianak,ID,-0.0333,109.3333,85.98,74,75,17.27,broken clouds
...,...,...,...,...,...,...,...,...,...,...
670,670,Laguna,US,38.4210,-121.4238,75.34,55,0,6.91,clear sky
674,674,Dwarka,IN,22.2394,68.9678,84.42,58,2,8.57,clear sky
681,681,Massakory,TD,12.9960,15.7293,83.53,11,8,6.62,clear sky
683,683,Yinchuan,CN,38.4681,106.2731,79.36,6,0,13.06,clear sky


In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_city_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Padang,ID,86.05,overcast clouds,-0.9492,100.3543,
9,Cockburn Town,TC,78.19,overcast clouds,21.4612,-71.1419,
11,Sao Joao Da Barra,BR,75.72,clear sky,-21.6403,-41.0511,
15,Hithadhoo,MV,83.34,overcast clouds,-0.6000,73.0833,
17,Pontianak,ID,85.98,broken clouds,-0.0333,109.3333,
23,Codrington,AU,76.30,broken clouds,-38.2667,141.9667,
24,Bambous Virieux,MU,79.21,light rain,-20.3428,57.7575,
25,New Norfolk,AU,76.66,few clouds,-42.7826,147.0587,
26,Panama City,PA,80.29,few clouds,8.9936,-79.5197,
27,Stabat,ID,80.80,overcast clouds,3.7610,98.4513,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")          

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.

c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Padang,ID,86.05,overcast clouds,-0.9492,100.3543,Mercure Padang
9,Cockburn Town,TC,78.19,overcast clouds,21.4612,-71.1419,Osprey Beach Hotel
11,Sao Joao Da Barra,BR,75.72,clear sky,-21.6403,-41.0511,Pousada Porto De Canoas
15,Hithadhoo,MV,83.34,overcast clouds,-0.6000,73.0833,Scoop Guest House
17,Pontianak,ID,85.98,broken clouds,-0.0333,109.3333,Aston Pontianak Hotel & Convention Center
...,...,...,...,...,...,...,...
668,Moree,AU,82.42,broken clouds,-29.4667,149.8500,Molika Springs Motel
670,Laguna,US,75.34,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
674,Dwarka,IN,84.42,clear sky,22.2394,68.9678,Hotel Narayan Avezika Comfort
683,Yinchuan,CN,79.36,clear sky,38.4681,106.2731,JJ Inns - Yinchuan New Moon Square Branch


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]


# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))